# Load Tensor Flow Lite Test Model And Run Inference on Test Data
Test Model and Data copied from here
https://jira01.corp.linkedin.com:8443/browse/MID-5889?focusedCommentId=22945746&page=com.atlassian.jira.plugin.system.issuetabpanels%3Acomment-tabpanel#comment-22945746

In [19]:
import numpy as np
import tensorflow as tf
import pandas as pd

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="native_tflite_model_from_mid_dummy_data_20220621.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load test data - note: Test data includes a prediction column already.
# Compare our inference with the included inference
df = pd.read_json("test_data_20220621.txt", orient="records", lines=True)
columns_to_drop = ['label']
df.drop(columns=columns_to_drop, inplace=True)
predictions = df.pop("prediction")

# infer score for each test and write scores to file
f = open("python-inferred-results.txt", "w")
for i in range(len(df)):
  input_data = np.array(df.loc[[i]], dtype= np.float32)
  interpreter.set_tensor(input_details[0]['index'], input_data)
  interpreter.invoke()
  output_data = interpreter.get_tensor(output_details[0]['index'])
  inferred = str(np.float32(predictions.loc[i]))
  print ("Test ", i, ", predicted ", output_data[0][0], ", inferred ", inferred)
  f.write(inferred+"\n")
f.close()


Test  0 , predicted  0.3915435 , inferred  0.39154354
Test  1 , predicted  0.3511033 , inferred  0.3511033
Test  2 , predicted  0.36121827 , inferred  0.36121827
Test  3 , predicted  0.35905254 , inferred  0.35905257
Test  4 , predicted  0.4028941 , inferred  0.40289408
Test  5 , predicted  0.4028941 , inferred  0.40289408
Test  6 , predicted  0.4028941 , inferred  0.40289408
Test  7 , predicted  0.4010978 , inferred  0.4010978
Test  8 , predicted  0.3758979 , inferred  0.37589782
Test  9 , predicted  0.39340648 , inferred  0.39340648
Test  10 , predicted  0.3595854 , inferred  0.3595854
Test  11 , predicted  0.2972039 , inferred  0.29720387
Test  12 , predicted  0.42317924 , inferred  0.42317924
Test  13 , predicted  0.4150917 , inferred  0.4150917
Test  14 , predicted  0.36539057 , inferred  0.36539057
Test  15 , predicted  0.37477267 , inferred  0.37477267
Test  16 , predicted  0.4476149 , inferred  0.44761485
Test  17 , predicted  0.4318053 , inferred  0.4318053
Test  18 , predicte